# 🚀 Watsonx.ai + LangGraph: Multi-Agent Systems Tutorial

## Introduction

Welcome to this comprehensive tutorial on building **Multi-Agent Systems** using **IBM watsonx.ai** and **LangGraph**!

### What You'll Learn:

1. **LangGraph Fundamentals** - Understanding state graphs and workflows
2. **Watsonx.ai Integration** - Connecting IBM's powerful LLMs
3. **Single Agent Systems** - Building your first LangGraph agent
4. **Multi-Agent Systems** - Creating collaborative AI agents
5. **Real-World Applications** - Practical use cases

### Prerequisites:

- IBM Cloud account with watsonx.ai access
- API Key and Project ID from watsonx.ai

---

## 📚 Table of Contents

1. [Setup & Installation](#setup)
2. [LangGraph Basics](#basics)
3. [Single Agent Example](#single)
4. [Multi-Agent Systems](#multi)
5. [Advanced Patterns](#advanced)

---

## 1. Setup & Installation <a id="setup"></a>

First, let's install all required packages for this tutorial.

In [ ]:
# Install required packages
!pip install -q langgraph langchain langchain-ibm python-dotenv

### Configure Your Credentials

For Google Colab, we'll use manual input. In production, use environment variables or `.env` files.

In [ ]:
import os
import getpass

# Set your credentials here
# Get your credentials from: https://cloud.ibm.com/iam/apikeys

if "WATSONX_API_KEY" not in os.environ:
    os.environ["WATSONX_API_KEY"] = getpass.getpass("Enter your Watsonx API Key: ")

if "WATSONX_URL" not in os.environ:
    os.environ["WATSONX_URL"] = input("Enter your Watsonx URL (default: https://us-south.ml.cloud.ibm.com): ") or "https://us-south.ml.cloud.ibm.com"

if "PROJECT_ID" not in os.environ:
    os.environ["PROJECT_ID"] = getpass.getpass("Enter your Project ID: ")

print("✅ Credentials configured successfully!")

---

## 2. LangGraph Basics <a id="basics"></a>

### What is LangGraph?

**LangGraph** is a library for building stateful, multi-actor applications with LLMs. It extends LangChain with:

- **State Management**: Maintain conversation context across nodes
- **Graph-Based Workflows**: Define complex agent interactions
- **Conditional Edges**: Dynamic routing based on state
- **Multi-Agent Orchestration**: Coordinate multiple AI agents

### Key Concepts:

1. **Nodes**: Functions that process the state
2. **Edges**: Connections between nodes
3. **State**: Shared data structure passed between nodes
4. **Graph**: The workflow definition

### Architecture Diagram:

```
START → Node 1 → Node 2 → Node 3 → END
          ↓        ↓        ↓
        State    State    State
```

---

## 3. Single Agent Example <a id="single"></a>

Let's start with a simple single-agent example using watsonx.ai and LangGraph.

### 3.1 Initialize Watsonx LLM

In [ ]:
from langchain_ibm import WatsonxLLM
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, Sequence
from langchain.schema import HumanMessage, AIMessage, BaseMessage
import operator

# Load credentials
api_key = os.getenv("WATSONX_API_KEY")
url = os.getenv("WATSONX_URL")
project_id = os.getenv("PROJECT_ID")

# Initialize Watsonx LLM
model_id = "ibm/granite-13b-instruct-v2"

watsonx_llm = WatsonxLLM(
    model_id=model_id,
    url=url,
    apikey=api_key,
    project_id=project_id,
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 300,
        "temperature": 0.7
    }
)

print("✅ Watsonx LLM initialized successfully!")

### 3.2 Define State Structure

The state holds the conversation history and any metadata.

In [ ]:
# Define the state structure
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    
print("✅ State structure defined!")

### 3.3 Create Simple Agent Node

In [ ]:
def simple_agent(state: AgentState) -> AgentState:
    """
    A simple agent that processes messages and generates responses.
    """
    messages = state["messages"]
    
    # Get the last user message
    last_message = messages[-1].content
    
    # Generate response using Watsonx
    response = watsonx_llm.invoke(last_message)
    
    # Return updated state
    return {"messages": [AIMessage(content=response)]}

print("✅ Simple agent node created!")

### 3.4 Build and Run the Graph

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)

# Add the agent node
workflow.add_node("agent", simple_agent)

# Set entry point
workflow.set_entry_point("agent")

# Add edge to END
workflow.add_edge("agent", END)

# Compile the graph
app = workflow.compile()

print("✅ Graph compiled successfully!")
print("\n📊 Graph structure:")
print("START → agent → END")

### 3.5 Test the Single Agent

In [ ]:
# Test the agent
inputs = {"messages": [HumanMessage(content="Tell me a joke about AI.")]}
result = app.invoke(inputs)

print("🤖 Agent Response:")
print("="*50)
print(result["messages"][-1].content)
print("="*50)

---

## 4. Multi-Agent Systems <a id="multi"></a>

Now let's build a **multi-agent system** where different agents collaborate to solve complex tasks.

### Use Case: Research Team

We'll create three specialized agents:
1. **Researcher**: Gathers information
2. **Analyst**: Analyzes the information
3. **Writer**: Produces the final output

### Multi-Agent Architecture:

```
START → Researcher → Analyst → Writer → END
          ↓           ↓         ↓
        State       State     State
```

### 4.1 Define Enhanced State

For multi-agent systems, we need a richer state structure.

In [ ]:
from typing import List, Dict

class MultiAgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    topic: str
    research_findings: str
    analysis: str
    final_report: str
    current_agent: str

print("✅ Multi-agent state structure defined!")

### 4.2 Create Specialized Agents

In [ ]:
def researcher_agent(state: MultiAgentState) -> MultiAgentState:
    """
    Researcher Agent: Gathers information about the topic.
    """
    topic = state.get("topic", "AI")
    
    prompt = f"""You are a research specialist. Provide 3-4 key facts about: {topic}
    Focus on recent developments and important concepts.
    Format your response as bullet points."""
    
    research = watsonx_llm.invoke(prompt)
    
    print("\n🔍 Researcher Agent completed its task")
    
    return {
        "messages": [AIMessage(content=f"Research completed on: {topic}")],
        "research_findings": research,
        "current_agent": "researcher"
    }


def analyst_agent(state: MultiAgentState) -> MultiAgentState:
    """
    Analyst Agent: Analyzes the research findings.
    """
    research = state.get("research_findings", "")
    topic = state.get("topic", "AI")
    
    prompt = f"""You are a data analyst. Analyze these research findings about {topic}:
    
    {research}
    
    Provide:
    1. Key insights
    2. Trends or patterns
    3. Implications
    
    Keep your analysis concise."""
    
    analysis = watsonx_llm.invoke(prompt)
    
    print("\n📊 Analyst Agent completed its task")
    
    return {
        "messages": [AIMessage(content=f"Analysis completed on research findings")],
        "analysis": analysis,
        "current_agent": "analyst"
    }


def writer_agent(state: MultiAgentState) -> MultiAgentState:
    """
    Writer Agent: Creates the final report.
    """
    research = state.get("research_findings", "")
    analysis = state.get("analysis", "")
    topic = state.get("topic", "AI")
    
    prompt = f"""You are a technical writer. Create a concise report about {topic} using:
    
    Research Findings:
    {research}
    
    Analysis:
    {analysis}
    
    Write a clear, professional summary in 2-3 paragraphs."""
    
    final_report = watsonx_llm.invoke(prompt)
    
    print("\n✍️  Writer Agent completed its task")
    
    return {
        "messages": [AIMessage(content=f"Final report completed")],
        "final_report": final_report,
        "current_agent": "writer"
    }

print("✅ All specialized agents created!")

### 4.3 Build Multi-Agent Workflow

In [ ]:
# Create the multi-agent graph
multi_agent_workflow = StateGraph(MultiAgentState)

# Add all agent nodes
multi_agent_workflow.add_node("researcher", researcher_agent)
multi_agent_workflow.add_node("analyst", analyst_agent)
multi_agent_workflow.add_node("writer", writer_agent)

# Set entry point
multi_agent_workflow.set_entry_point("researcher")

# Define the workflow edges
multi_agent_workflow.add_edge("researcher", "analyst")
multi_agent_workflow.add_edge("analyst", "writer")
multi_agent_workflow.add_edge("writer", END)

# Compile the multi-agent graph
multi_agent_app = multi_agent_workflow.compile()

print("✅ Multi-agent workflow compiled successfully!")
print("\n📊 Multi-Agent Graph Structure:")
print("START → Researcher → Analyst → Writer → END")

### 4.4 Run Multi-Agent System

In [ ]:
# Run the multi-agent system
topic = "Quantum Computing"

print(f"\n🚀 Starting Multi-Agent Research on: {topic}")
print("="*60)

initial_state = {
    "messages": [HumanMessage(content=f"Research and analyze {topic}")],
    "topic": topic,
    "research_findings": "",
    "analysis": "",
    "final_report": "",
    "current_agent": "start"
}

result = multi_agent_app.invoke(initial_state)

print("\n" + "="*60)
print("✅ Multi-Agent Process Complete!")
print("="*60)

### 4.5 Display Results

In [ ]:
print("\n" + "="*60)
print("📋 RESEARCH FINDINGS")
print("="*60)
print(result.get("research_findings", "No findings"))

print("\n" + "="*60)
print("📊 ANALYSIS")
print("="*60)
print(result.get("analysis", "No analysis"))

print("\n" + "="*60)
print("📄 FINAL REPORT")
print("="*60)
print(result.get("final_report", "No report"))
print("="*60)

---

## 5. Advanced Patterns <a id="advanced"></a>

### 5.1 Conditional Routing

Let's create a system where agents can make decisions about workflow routing.

In [ ]:
from langgraph.graph import END

class RoutingState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    question_type: str
    response: str


def classifier_agent(state: RoutingState) -> RoutingState:
    """
    Classifies the type of question.
    """
    question = state["messages"][-1].content
    
    prompt = f"""Classify this question into ONE category: 'technical', 'creative', or 'general'
    Question: {question}
    
    Respond with ONLY the category word, nothing else."""
    
    classification = watsonx_llm.invoke(prompt).strip().lower()
    
    # Ensure valid classification
    if classification not in ['technical', 'creative', 'general']:
        classification = 'general'
    
    print(f"\n🎯 Question classified as: {classification}")
    
    return {
        "messages": [AIMessage(content=f"Classified as {classification}")],
        "question_type": classification
    }


def technical_agent(state: RoutingState) -> RoutingState:
    """
    Handles technical questions.
    """
    question = state["messages"][0].content
    prompt = f"""As a technical expert, answer this question with precision:
    {question}"""
    
    response = watsonx_llm.invoke(prompt)
    print("\n🔧 Technical Agent responded")
    
    return {
        "messages": [AIMessage(content=response)],
        "response": response
    }


def creative_agent(state: RoutingState) -> RoutingState:
    """
    Handles creative questions.
    """
    question = state["messages"][0].content
    prompt = f"""As a creative writer, answer this question imaginatively:
    {question}"""
    
    response = watsonx_llm.invoke(prompt)
    print("\n🎨 Creative Agent responded")
    
    return {
        "messages": [AIMessage(content=response)],
        "response": response
    }


def general_agent(state: RoutingState) -> RoutingState:
    """
    Handles general questions.
    """
    question = state["messages"][0].content
    prompt = f"""Answer this general question clearly and concisely:
    {question}"""
    
    response = watsonx_llm.invoke(prompt)
    print("\n💬 General Agent responded")
    
    return {
        "messages": [AIMessage(content=response)],
        "response": response
    }


def route_question(state: RoutingState) -> str:
    """
    Routes to the appropriate agent based on classification.
    """
    question_type = state.get("question_type", "general")
    
    routing_map = {
        "technical": "technical_agent",
        "creative": "creative_agent",
        "general": "general_agent"
    }
    
    return routing_map.get(question_type, "general_agent")


print("✅ Conditional routing agents created!")

### 5.2 Build Routing Workflow

In [ ]:
# Create routing workflow
routing_workflow = StateGraph(RoutingState)

# Add nodes
routing_workflow.add_node("classifier", classifier_agent)
routing_workflow.add_node("technical_agent", technical_agent)
routing_workflow.add_node("creative_agent", creative_agent)
routing_workflow.add_node("general_agent", general_agent)

# Set entry point
routing_workflow.set_entry_point("classifier")

# Add conditional edges
routing_workflow.add_conditional_edges(
    "classifier",
    route_question,
    {
        "technical_agent": "technical_agent",
        "creative_agent": "creative_agent",
        "general_agent": "general_agent"
    }
)

# Add edges to END
routing_workflow.add_edge("technical_agent", END)
routing_workflow.add_edge("creative_agent", END)
routing_workflow.add_edge("general_agent", END)

# Compile
routing_app = routing_workflow.compile()

print("✅ Routing workflow compiled!")
print("\n📊 Routing Graph Structure:")
print("""START → Classifier → [Technical Agent]
                    → [Creative Agent]
                    → [General Agent] → END""")

### 5.3 Test Conditional Routing

In [ ]:
# Test with different question types
test_questions = [
    "Explain how neural networks work",
    "Write a short poem about clouds",
    "What is the capital of France?"
]

for question in test_questions:
    print("\n" + "="*60)
    print(f"❓ Question: {question}")
    print("="*60)
    
    result = routing_app.invoke({
        "messages": [HumanMessage(content=question)],
        "question_type": "",
        "response": ""
    })
    
    print(f"\n💡 Response:\n{result['response']}")
    print("="*60)

---

## 6. Real-World Use Cases

### Customer Support System

Let's build a practical customer support multi-agent system.

In [ ]:
class SupportState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    customer_query: str
    issue_category: str
    solution: str
    escalation_needed: bool


def triage_agent(state: SupportState) -> SupportState:
    """
    Triages customer issues.
    """
    query = state["customer_query"]
    
    prompt = f"""Categorize this customer issue: {query}
    
    Categories: billing, technical, account, general
    Respond with ONLY the category."""
    
    category = watsonx_llm.invoke(prompt).strip().lower()
    
    print(f"\n📞 Triage: Issue categorized as '{category}'")
    
    return {
        "messages": [AIMessage(content=f"Issue triaged")],
        "issue_category": category
    }


def resolution_agent(state: SupportState) -> SupportState:
    """
    Provides solutions to customer issues.
    """
    query = state["customer_query"]
    category = state["issue_category"]
    
    prompt = f"""Provide a helpful solution for this {category} issue:
    {query}
    
    Be specific and actionable."""
    
    solution = watsonx_llm.invoke(prompt)
    
    print("\n✅ Resolution: Solution provided")
    
    return {
        "messages": [AIMessage(content="Solution generated")],
        "solution": solution,
        "escalation_needed": False
    }


# Build support workflow
support_workflow = StateGraph(SupportState)
support_workflow.add_node("triage", triage_agent)
support_workflow.add_node("resolution", resolution_agent)

support_workflow.set_entry_point("triage")
support_workflow.add_edge("triage", "resolution")
support_workflow.add_edge("resolution", END)

support_app = support_workflow.compile()

print("\n✅ Customer Support System ready!")

### Test Customer Support System

In [ ]:
# Test the support system
customer_issue = "I can't log into my account after resetting my password"

print(f"\n🎫 Customer Issue: {customer_issue}")
print("="*60)

support_result = support_app.invoke({
    "messages": [HumanMessage(content=customer_issue)],
    "customer_query": customer_issue,
    "issue_category": "",
    "solution": "",
    "escalation_needed": False
})

print("\n" + "="*60)
print("📋 SUPPORT SOLUTION")
print("="*60)
print(support_result["solution"])
print("="*60)

---

## 7. Best Practices & Tips

### ✅ Do's:

1. **Clear State Definition**: Define explicit state structures for your agents
2. **Specialized Agents**: Create focused agents with specific responsibilities
3. **Error Handling**: Add try-catch blocks for robust production systems
4. **Logging**: Track agent execution for debugging
5. **Testing**: Test individual agents before integrating

### ❌ Don'ts:

1. **Avoid Circular Dependencies**: Ensure your graph has clear flow
2. **Don't Overload Agents**: Keep agent responsibilities focused
3. **Don't Ignore State**: Always pass complete state information

### 🎯 Performance Tips:

- Use appropriate model parameters (temperature, max_tokens)
- Cache results when possible
- Implement parallel execution for independent agents
- Monitor token usage

---

## 8. Conclusion & Next Steps

### What We've Learned:

1. ✅ LangGraph fundamentals and architecture
2. ✅ Integrating watsonx.ai with LangGraph
3. ✅ Building single-agent systems
4. ✅ Creating multi-agent workflows
5. ✅ Implementing conditional routing
6. ✅ Real-world applications

### Next Steps:

1. **Experiment**: Try different agent configurations
2. **Extend**: Add more specialized agents
3. **Deploy**: Move to production with proper error handling
4. **Optimize**: Fine-tune prompts and parameters
5. **Scale**: Implement parallel execution and caching

### Resources:

- [LangGraph Documentation](https://python.langchain.com/docs/langgraph)
- [Watsonx.ai Documentation](https://www.ibm.com/docs/en/watsonx-as-a-service)
- [LangChain IBM Integration](https://python.langchain.com/docs/integrations/providers/ibm)

---

## 🎉 Congratulations!

You've completed the Watsonx.ai + LangGraph Multi-Agent Tutorial!

Feel free to experiment with the code and build your own multi-agent systems.

---

### 📝 Practice Exercise

Try creating your own multi-agent system for:
- Content moderation pipeline
- Data analysis workflow
- Educational tutoring system
- Code review automation

**Happy Building! 🚀**